Você faz parte da equipe de Analytics de uma grande marca de vestuário com mais de 25 lojas espalhadas em Shoppings de todo o Brasil.

Toda semana você precisa enviar para a diretoria um ranking atualizado com as 25 lojas contendo 3 informações:

Faturamento de cada Loja;  
Quantidade de produtos vendidos de cada Loja;  
Ticket médio dos produto de cada Loja.  

Além disso, cada loja tem 1 gerente que precisa receber o resumo das informações da loja dele. Por isso, cada gerente deve receber no e-mail:  

Faturamento da sua Loja;  
Quantidade de produtos Vendidos da sua Loja;  
Ticket médio dos produto da sua Loja.  
Esse relatório é sempre enviado como um resumo de todos os dados disponíveis no ano.  

In [1]:
import os
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText


In [2]:
df = pd.read_excel('vendas.xlsx')
df

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,Iguatemi Esplanada,Sapato Estampa,1,358,358
1,1,2019-01-01,Iguatemi Esplanada,Camiseta,2,180,360
2,1,2019-01-01,Iguatemi Esplanada,Sapato Xadrez,1,368,368
3,2,2019-01-02,Norte Shopping,Relógio,3,200,600
4,2,2019-01-02,Norte Shopping,Chinelo Liso,1,71,71
...,...,...,...,...,...,...,...
100994,69996,2019-12-26,Center Shopping Uberlândia,Short Listrado,2,102,204
100995,69996,2019-12-26,Center Shopping Uberlândia,Mochila,4,270,1080
100996,69996,2019-12-26,Center Shopping Uberlândia,Pulseira Estampa,1,87,87
100997,69997,2019-12-26,Ribeirão Shopping,Camisa Listrado,1,108,108


In [3]:
df_lojas = df.copy()

#Renomear colunas
renomear_colunas = {
    'Código Venda' : 'codigo_venda', 
    'Data' : 'data', 
    'ID Loja' : 'id_loja', 
    'Produto' : 'produto', 
    'Quantidade' : 'quantidade',
    'Valor Unitário' : 'valor_unitario', 
    'Valor Final' : 'valor_final'
}

df_lojas = df_lojas.rename(columns=renomear_colunas)


In [4]:

df_resumo = df_lojas.groupby('id_loja').agg({
    'valor_final': 'sum',     
    'quantidade': 'sum'       
})

# Calcular ticket médio
df_resumo['ticket_medio'] = df_resumo['valor_final'] / df_resumo['quantidade']

# transformar 'id_loja' em coluna
df_resumo = df_resumo.reset_index()

df_resumo


,id_loja,valor_final,quantidade,ticket_medio
0,Bourbon Shopping SP,1726110,8863,194.754598
1,Center Shopping Uberlândia,1668921,8627,193.453228
2,Iguatemi Campinas,1762419,8935,197.248909
3,Iguatemi Esplanada,1699681,8580,198.098019
4,Norte Shopping,1711968,9014,189.923231
5,Novo Shopping Ribeirão Preto,1678225,8751,191.775226
6,Palladium Shopping Curitiba,1721120,9091,189.321307
7,Parque Dom Pedro Shopping,1631630,8388,194.519552
8,Passei das Águas Shopping,1649014,8618,191.345324
9,Ribeirão Shopping,1707122,8825,193.441586


In [ ]:
email_gerentes = {
    'Shopping Eldorado' : 'syafonseca@gmail.com'
}

def enviar_email(destinatario, assunto, corpo):
    smtp_host = 'smtp.gmail.com'
    smtp_port = 587
    email_remetente = 'syafonseca@gmail.com'
    senha_app = "dfgr wfgm dtyoq qylx"



    msg = MIMEMultipart()
    msg['From'] = email_remetente
    msg['To'] = destinatario
    msg['subject'] = assunto

    msg.attach(MIMEText(corpo, 'plain'))


    with smtplib.SMTP(smtp_host, smtp_port) as server:
        server.starttls()
        server.login(email_remetente, senha_app)
        server.send_message(msg)
    print(f'Email Enviado para {destinatario}')

for _, row in df_resumo.iterrows():
    loja = row['id_loja']
    email_gerente = email_gerentes.get(loja, '').strip()

    if email_gerente:
        assunto = f'Relatório Anual da Loja {loja}'
        corpo = f"""
Senhor Gerente da filial no {loja}:

Segue resumo anual:

- Faturamento: R${row['valor_final']:,.2f}
- Quantidade de Produtos Vendidos: {row['quantidade']:,.0f}
- Ticket Médio das Compras no Período: R${row['ticket_medio']:,.2f}

Atenciosamente,
Equipe de Analytics
"""
        enviar_email(email_gerente, assunto, corpo)
    else:
        print(f'E-mail do gerente para a loja {loja} não encontrado')


E-mail do gerente para a loja Bourbon Shopping SP não encontrado
E-mail do gerente para a loja Center Shopping Uberlândia não encontrado
E-mail do gerente para a loja Iguatemi Campinas não encontrado
E-mail do gerente para a loja Iguatemi Esplanada não encontrado
E-mail do gerente para a loja Norte Shopping não encontrado
E-mail do gerente para a loja Novo Shopping Ribeirão Preto não encontrado
E-mail do gerente para a loja Palladium Shopping Curitiba não encontrado
E-mail do gerente para a loja Parque Dom Pedro Shopping não encontrado
E-mail do gerente para a loja Passei das Águas Shopping não encontrado
E-mail do gerente para a loja Ribeirão Shopping não encontrado
E-mail do gerente para a loja Rio Mar Recife não encontrado
E-mail do gerente para a loja Rio Mar Shopping Fortaleza não encontrado
E-mail do gerente para a loja Salvador Shopping não encontrado
E-mail do gerente para a loja Shopping Barra não encontrado
E-mail do gerente para a loja Shopping Center Interlagos não encontra